In [1]:
print('start ch uploads')
#Clickhouse db w/ Goldsky
# https://clickhouse.com/docs/en/integrations/python

import requests as r
import pandas as pd
import clickhouse_connect as cc
import os

import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
import pandas_utils as p
import clickhouse_utils as ch
import csv_utils as cu
import google_bq_utils as bqu
import opstack_metadata_utils as ops
import goldsky_db_utils as gsb
sys.path.pop()

import time

start ch uploads


In [2]:
client = ch.connect_to_clickhouse_db() #Default is OPLabs DB
# client.close()

table_name = 'daily_aggegate_l2_chain_usage_goldsky'

In [3]:
# bqu.delete_bq_table('api_table_uploads',table_name)

In [4]:
trailing_days = 7
max_execution_secs = 3000

In [5]:
# chain_mappings_list = [
#     # {'schema_name': 'zora', 'display_name': 'Zora', 'has_blob_fields': True},
#     # {'schema_name': 'pgn', 'display_name': 'Public Goods Network', 'has_blob_fields': True},
#     # {'schema_name': 'base', 'display_name': 'Base', 'has_blob_fields': False},
#     # {'schema_name': 'op', 'display_name': 'OP Mainnet', 'has_blob_fields': True},
#     {'schema_name': 'mode', 'display_name': 'Mode', 'has_blob_fields': True},
#     {'schema_name': 'metal', 'display_name': 'Metal', 'has_blob_fields': True},
#     {'schema_name': 'fraxtal', 'display_name': 'Fraxtal', 'has_blob_fields': True},
#     # {'schema_name': 'bob', 'display_name': 'BOB (Build on Bitcoin)', 'has_blob_fields': False},
#     {'schema_name': 'cyber', 'display_name': 'Cyber', 'has_blob_fields': True},
#     {'schema_name': 'mint', 'display_name': 'Mint', 'has_blob_fields': True},
#     # Add more mappings as needed
# ]
# chain_mappings_dict = {item['schema_name']: item['display_name'] for item in chain_mappings_list}


In [6]:
# Get Chain List
chain_configs = ops.get_superchain_metadata_by_data_source('oplabs') # OPLabs db

# if we want to filter for backfills
# chain_configs = chain_configs[chain_configs['blockchain'].isin(['swan','lisk'])]

chain_configs

,chain_name,display_name,mainnet_chain_id,chain_layer,alignment,da_layer,output_root_layer,gas_token,block_time_sec,blockchain
0,op,OP Mainnet,10.0,L2,OP Chain,ethereum,ethereum,ETH,2,op
1,zora,Zora,7777777.0,L2,OP Chain,ethereum,ethereum,ETH,2,zora
2,base,Base,8453.0,L2,OP Chain,ethereum,ethereum,ETH,2,base
3,mode,Mode,34443.0,L2,OP Chain,ethereum,ethereum,ETH,2,mode
5,lisk,Lisk,1135.0,L2,OP Chain,ethereum,ethereum,ETH,2,lisk
6,metal,Metal,1750.0,L2,OP Chain,ethereum,ethereum,ETH,2,metal
7,mint,Mint,185.0,L2,OP Chain,ethereum,ethereum,ETH,2,mint
13,fraxtal,Fraxtal,252.0,L2,OP Chain,fraxtalda,ethereum,frxETH,2,fraxtal
14,redstone,Redstone,690.0,L2,OP Chain,op-plasma,ethereum,ETH,2,redstone
15,cyber,Cyber,7560.0,L2,OP Chain,eigenda,ethereum,ETH,2,cyber


In [7]:
sql_directory = "inputs/sql/"

query_names = [
        # Must match the file name in inputs/sql
        "ch_template_alltime_chain_activity"
]

In [8]:
# Set the memory limit using the settings parameter
settings = {
    'max_memory_usage': 200000000000  # 200GB in bytes
}


In [9]:
unified_dfs = []
for qn in query_names:
        for index, chain in chain_configs.iterrows():
                chain_schema = chain['blockchain']
                display_name = chain['display_name']
                block_time_sec = chain['block_time_sec']
                # Read the SQL query from file
                with open(os.path.join(sql_directory, f"{qn}.sql"), "r") as file:
                        query = file.read()
                print(qn + ' - ' + chain_schema)

                #Pass in Params to the query
                query = query.replace("@chain_db_name@", chain_schema)
                query = query.replace("@trailing_days@", str(trailing_days))
                query = query.replace("@block_time_sec@", str(block_time_sec))
                query = query.replace("@max_execution_secs@", str(max_execution_secs))


                query = gsb.process_goldsky_sql(query)

                #Save the query
                output_folder = os.path.join("outputs", "sql")
                os.makedirs(output_folder, exist_ok=True)
                filename = f"{chain_schema}_query.sql"
                file_path = os.path.join(output_folder, filename)
                with open(file_path, 'w') as file:
                        file.write(query)

                # Execute the query
                result_df = client.query_df(query, settings=settings)
        #         # Write to csv
        #         df.to_csv('outputs/chain_data/' + qn + '.csv', index=False)
        #         # print(df.sample(5))
        #         time.sleep(1)
                
                result_df['chain_raw'] = result_df['chain']
                result_df['chain'] = display_name
                unified_dfs.append(result_df)

        write_df = pd.concat(unified_dfs)
        write_df.to_csv('outputs/chain_data/' + qn + '.csv', index=False)
        
        # # # Print the results

ch_template_alltime_chain_activity - op


DatabaseError: :HTTPDriver for https://pdmv9lhojy.us-west-2.aws.clickhouse.cloud:8443 returned response code 400)
 Code: 62. DB::Exception: Syntax error (Multi-statements are not allowed): failed at position 36 (end of query) (line 1, col 36): ; -- Set to 200GB

SELECT *
    , num_blocks * block_time_sec as active_secs_per_day
    
    , l2_num_txs_per_day / num_blocks AS l2_num_txs_per_day_per_block
. . (SYNTAX_ERROR) (version 24.5.1.22957 (official build))


In [ ]:
# write_df.dtypes
write_df['chain_id'] = write_df['chain_id'].astype('int64')

In [ ]:
#BQ Upload
time.sleep(1)
bqu.append_and_upsert_df_to_bq_table(write_df, table_name, unique_keys = ['dt','chain','network'])

In [ ]:
# Copy to Dune
print('upload bq to dune')
sql = '''
SELECT *
FROM `api_table_uploads.daily_aggegate_l2_chain_usage_goldsky`
'''
bq_df = bqu.run_query_to_df(sql)

dune_table_name = 'ch_template_alltime_chain_activity'
d.write_dune_api_from_pandas(bq_df, dune_table_name,table_description = dune_table_name)